# Identificação em tempo real de objeto pela cor

### Carregando os pacotes necessários

In [1]:
import numpy as np
import cv2

### Captura do vídeo para encontrar o intervalo de cores

In [6]:
# O zero corresponde à câmera selecionada
cap = cv2.VideoCapture(0)

def nothing(x):
    pass

# Criar trackbar
windowName = "Imagem"
cv2.namedWindow(windowName)
cv2.createTrackbar("lowerH", windowName, 0, 255, nothing)
cv2.createTrackbar("lowerS", windowName, 0, 255, nothing)
cv2.createTrackbar("lowerV", windowName, 0, 255, nothing)
cv2.createTrackbar("upperH", windowName, 255, 255, nothing)
cv2.createTrackbar("upperS", windowName, 255, 255, nothing)
cv2.createTrackbar("upperV", windowName, 255, 255, nothing)

while(1):
    _, frame = cap.read()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    lH = cv2.getTrackbarPos("lowerH", windowName)
    lS = cv2.getTrackbarPos("lowerS", windowName)
    lV = cv2.getTrackbarPos("lowerV", windowName)
    lower_val = np.array([lH,lS,lV])
    
    uH = cv2.getTrackbarPos("upperH", windowName)
    uS = cv2.getTrackbarPos("upperS", windowName)
    uV = cv2.getTrackbarPos("upperV", windowName)
    upper_val = np.array([uH,uS,uV])
    
    mask = cv2.inRange(hsv, lower_val, upper_val)
    hsvFilt = cv2.bitwise_and(frame, frame, mask=mask)
    
    cv2.imshow(windowName, frame)
    cv2.imshow("Result", hsvFilt)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()
cap.release()

### Encontrando a circunferência

In [7]:
cap = cv2.VideoCapture(0)

while(1):
    _, frame = cap.read()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Valores determinados anteriormente
    lower_val = np.array([20,110,0])
    upper_val = np.array([40,255,255])
    
    mask = cv2.inRange(hsv, lower_val, upper_val)
    
# Descomentar caso seja necessário aplicar operação morfológica    
#    kernel = np.ones((1,1), np.uint8)
#    maskClosing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    # Encontrando os contornos
    _,cnts,_ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Encontrando o círculo de maior área
    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        center = (int(x), int(y))
    
    # Desenhando a circunferência e o centro
    cv2.circle(frame, center, int(radius), (0, 0, 255), 2)
    cv2.circle(frame, center, 1, (0, 0, 255), 5)
    cv2.imshow("Frame", frame)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
    
cv2.destroyAllWindows()